### Steps
- Use a list of **app_id** to get info from Steam crawler and insert **app_id**, **game_name**, **header_img_url**, **total_positive**, **total_negative**, **total_reviews** into table **games** and insert **game_id**, **review**, **recommended**, **time** into table **reviews**.
- Preprocess **review** in table **reviews** (add_missing_punct, replace_bullets, remove_url, remove_html_tags, normalize_single_quote, remove_non_ascii, remove_ansi_escape_sequences, remove_multi_whitespaces), then tokenize_sent and remove_leading_symbols to insert **review_id**, **sent** into table **sents**. 
- Preprocess **sent** in table **sents** (lowercase, expand contractions, remove_digits, remove_symbols, remove_multi_whitespaces, lemmatize_text, remove_stopwords) to create **sent_prep** in table **sents**.
- Use **sent_prep**, **review_id** in table **sents** to insert **review_prep** in table **reviews** by joining **sent_prep**.
- Use **review_prep** in table **reviews** to calculate special bigrams frequency, get 50 most frequent keywords.
- Insert **kw**, **freq** into table **kws**.
- Embed 50 keywords using S-BERT and cluster them using agglomerative clustering with a distance_threshold=0.6. 
	- Insert **cluster_name** (name of the most frequent keyword in cluster) into table **clusters**.
	- Insert **cluster_id** in table **kws**.
- Loop through **sent_prep** in table **sents**, fuzzy-match each **kw** in table **kws**. 
    - Insert **cluster_id**, **sent_id** in table **clusters_sents**t to link table **clusters** and **sents**.

### Step 1:

- Use a list of **app_id** to get info from Steam crawler and insert **app_id**, **game_name**, **header_img_url**, **total_positive**, **total_negative**, **total_reviews** into table **games** and insert **game_id** (fk), **review**, **recommended**, **time** into table **reviews**.

In [1]:
import requests 
import pandas as pd
import numpy as np

import download_steam_reviews

import sqlite3

import re
from bs4 import BeautifulSoup

import spacy
# nlp = spacy.load("en_core_web_sm")
nlp = spacy.load("en_core_web_trf")

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

from sentence_transformers import SentenceTransformer
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')

from sklearn.cluster import AgglomerativeClustering

from fuzzysearch import find_near_matches

from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import flair

sentiment_model = flair.models.TextClassifier.load('sentiment')
sentiment_model_fast = flair.models.TextClassifier.load('sentiment-fast')
senti_analyzer = SentimentIntensityAnalyzer()

import pickle

import plotly.express as px
import plotly.graph_objects as go

from sklearn.cluster import KMeans
import hdbscan
from summa.summarizer import summarize

from scipy.cluster.vq import vq

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap.umap_ as umap

from tqdm import tqdm

from collections import Counter

from fuzzywuzzy import fuzz, process

from spacy import displacy

from pprint import pprint

D:\Anaconda3\envs\steam_insider\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


2021-07-06 22:38:41,288 loading file C:\Users\HuyTran\.flair\models\sentiment-en-mix-distillbert_4.pt
2021-07-06 22:38:45,268 loading file C:\Users\HuyTran\.flair\models\sentiment-en-mix-ft-rnn.pt


In [2]:
# from contractions import contractions
contractions = {
    "ain't": "is not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "i'd": "i would",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it would",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "it will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she would",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as",
    "that'd": "that would",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there would",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they would",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you would",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [3]:
conn = sqlite3.connect('./data/steam_reviews.db') 
cursor = conn.cursor()

In [ ]:
def print_cursor(cursor): 
    print(cursor.execute("""
        select * from games;
    """).fetchall())

In [ ]:
print_cursor(cursor)

In [ ]:
print(cursor.execute("""
        select game_id from games 
        where app_id=367520;
    """).fetchall()[0][0])

In [ ]:
# 428550 - Momodora: Reverie under the Moonlight
# 367520 - Hollow Knight
app_ids = [428550, 367520]

In [ ]:
### get game_name ###

def get_app_list(): 
    app_list_url = 'https://api.steampowered.com/ISteamApps/GetAppList/v2/'
    resp_data = requests.get(app_list_url)
    return resp_data.json()

def get_name(app_id, app_list): 
    for app in app_list['applist']['apps']: 
        if app['appid'] == app_id: 
            return app['name']

In [ ]:
### get header_img_url  

def get_header_img_url(app_id): 
    return f'https://cdn.cloudflare.steamstatic.com/steam/apps/{app_id}/header.jpg'

In [ ]:
### get total_positive, total_negative, total_reviews in Crawler for table games 
### get game_id, review, recommended, time in Crawler for table games 

app_list = get_app_list()

request_params = {
    'language': 'english'
}

# load or download new (maximum ~5000 newest reviews)
load_mode = True

for app_id in app_ids: 
    game_tuples = [] 

    game_name = get_name(app_id, app_list)
    
    header_img_url = get_header_img_url(app_id)
    
    total_positive, total_negative, total_reviews = 0, 0, 0
    
    if load_mode: 
        review_dict = download_steam_reviews.load_review_dict(app_id)['reviews'].values()
    else: 
        review_dict = download_steam_reviews.download_reviews_for_app_id(app_id, 
                                                                     chosen_request_params=request_params, 
                                                                     reviews_limit=5000)[0]['reviews'].values()
    
    review_tuples = []
    
    with conn:
        cursor.execute("""INSERT INTO games (app_id, game_name, header_img_url) VALUES (?, ?, ?);""", 
                       (app_id, game_name, header_img_url))
    
    # get game_id (fk) for table reviews
    game_id = cursor.execute("""SELECT game_id FROM games WHERE app_id=?;""", (app_id,)).fetchone()[0] 
    
    for review_dict_value in review_dict: 
        total_reviews += 1
    
        voted_up = 1 if review_dict_value['voted_up'] else 0
    
        if voted_up: 
            total_positive += 1
        else: 
            total_negative += 1
     
        review = review_dict_value['review']
        recommended = voted_up
        time = review_dict_value['timestamp_updated']
        
        review_tuples.append((review, recommended, time, game_id))
    
    with conn:
        cursor.execute("""UPDATE games SET (total_positive, total_negative, total_reviews) = (?, ?, ?)
                            WHERE game_id=?;""",
                       (total_positive, total_negative, total_reviews, game_id))
    
    with conn:
        cursor.executemany("""INSERT INTO reviews (review, recommended, time, game_id) VALUES 
                                (?, ?, ?, ?);""", review_tuples)    

### Step 2:

- Preprocess **review** in table **reviews** (add_missing_punct, replace_bullets, remove_url, remove_html_tags, normalize_single_quote, remove_non_ascii, remove_ansi_escape_sequences, remove_multi_whitespaces), then tokenize_sent and remove_leading_symbols to insert **review_id**, **sent** into table **sents**. 
- Preprocess **sent** in table **sents** (lowercase, expand contractions, remove_digits, remove_symbols, remove_multi_whitespaces, lemmatize_text, remove_stopwords) to insert **sent_prep** in table **sents**.
- Use **sent_prep**, **review_id** in table **sents** to insert **review_prep** in table **reviews** by joining **sent_prep**.
        

    

In [ ]:
df_reviews = pd.read_sql_query("""SELECT review_id, review, game_id 
                    FROM reviews JOIN games USING(game_id);""", conn)

In [ ]:
df_reviews

In [274]:
def add_missing_punct(text): 
    return re.sub('([A-Za-z0-9])\s*$', '\g<1>. ', text)


def replace_bullets(text): 
    text = re.sub('([A-Za-z0-9])\s*\n+\s*[+-]?\s*', '\g<1>. ', text)
    text = re.sub('\s*([:+-]+)\s*\n+\s*[+-]?\s*', '. ', text) 
    return text
    
    
# remove url from text
def remove_url(text):
    return re.sub(r"http\S+", ' ', text)


# remove HTML tags
def remove_html_tags(text):
    soup = BeautifulSoup(text, "lxml")
    text = soup.get_text()
    # remove square brackets and characters inside
    text = re.sub('\[(.*?)\]', ' ', text)
    return text


# replace ’ with ' 
def normalize_single_quote(text):
    return re.sub('[’‘]', '\'', text)


# remove non english characters effectively
def remove_non_ascii(text): 
    return text.encode("ascii", errors="ignore").decode()
    
    
# remove ANSI escape sequences
def remove_ansi_escape_sequences(text):
    ansi_escape = re.compile(r'(?:\x1B[@-_]|[\x80-\x9F])[0-?]*[ -/]*[@-~]')
    return ansi_escape.sub('', text)
    
    
# remove multiple whitespaces with single whitespace
def remove_multi_whitespaces(text): 
    return re.sub('\s+', ' ', text.strip())

In [ ]:
def remove_bullet_nums(sent): 
    return re.sub('^\s*\d+\n*[.\)]+\s*([^\d])', '\g<1>', sent)

def remove_leading_symbols(sent):
    return re.sub('^[^A-Za-z\"\'\d]+', '', sent)

def uppercase_first(sent): 
    return sent[0].upper() + sent[1:] if len(sent) != 0 else sent

In [ ]:
def tokenize_sent(text):
    doc = nlp(text, disable=['ner', 'attribute_ruler', 'lemmatizer', 'sentencizer'])
    sents = [str(sent).strip() for sent in doc.sents]
    return sents

In [275]:
def lowercase(text):
    return text.lower()

def expand_contractions(text):
    for key in contractions:
        value = contractions[key]
        text = text.replace(key, value)
    return text

# remove digits 
def remove_digits(text): 
    return re.sub('\d+', ' ', text)

# remove symbols 
def remove_symbols(text):
    return re.sub('[^A-Za-z,.\s\d]+', ' ', text)

# lemmatization with spacy 
def lemmatize_text(text): 
    doc = nlp(text, disable=['parser','ner'])
    lemma = [token.lemma_ for token in doc if token.pos_ != 'PUNCT']
    return ' '.join(lemma)

# remove stop words 
def remove_stopwords(text, word_list=[]):
    stop_words = stopwords.words("english")
    stop_words.extend(word_list)
    stop_words = set(stop_words)
    return ' '.join(e.lower() for e in text.split() if e.lower() not in stop_words)

def get_extra_stopwords(game_name): 
    stopwords = set(['game', 'lot', 'bit', 'way '])
    doc = nlp(game_name, disable=['parser', 'ner'])
    for token in doc: 
        if token.pos_ not in {'PUNCT', 'NUM'}:
            stopwords.add(token.text.lower())
    return stopwords

In [ ]:
sent_tuples = []

for game_id in df_reviews['game_id'].unique():
    with conn: 
        game_name = cursor.execute("""SELECT game_name FROM games WHERE game_id=?;""", (int(game_id),)).fetchone()[0]
    
    extra_stopwords = get_extra_stopwords(game_name)
    
    df_reviews_game = df_reviews[df_reviews['game_id'] == game_id]
    
    reviews_game_cleaned = df_reviews_game['review'].map(add_missing_punct)\
                    .map(replace_bullets)\
                    .map(remove_url)\
                    .map(remove_html_tags)\
                    .map(normalize_single_quote)\
                    .map(remove_non_ascii)\
                    .map(remove_ansi_escape_sequences)\
                    .map(remove_multi_whitespaces)
    
    for review_id, review in zip(df_reviews_game['review_id'], reviews_game_cleaned):
        sents = pd.Series(tokenize_sent(review)).map(remove_bullet_nums)\
                                                .map(remove_leading_symbols)\
                                                .map(uppercase_first)\
                                                .map(add_missing_punct)\
                                                .map(remove_multi_whitespaces)

        sents_prep = sents.map(lowercase)\
                        .map(expand_contractions)\
                        .map(remove_digits)\
                        .map(remove_symbols)\
                        .map(remove_multi_whitespaces)\
                        .map(lemmatize_text)\
                        .map(lambda x: remove_stopwords(x, word_list=extra_stopwords))

        for sent, sent_prep in zip(sents, sents_prep):
            sent_tuples.append((review_id, sent, sent_prep))

In [ ]:
with conn:
    cursor.executemany("""INSERT INTO sents (review_id, sent, sent_prep) VALUES 
                        (?, ?, ?);""", sent_tuples)    

### Step 3: 
- Use **review_prep** in table **reviews** to calculate special bigrams frequency, get 50 most frequent keywords.
- Insert **kw**, **freq** into table **kws**.
- Embed 50 keywords using S-BERT and cluster them using agglomerative clustering with a distance_threshold=0.6. 
	- Insert **cluster_name** (name of the most frequent keyword in cluster) into table **clusters**.
	- Insert **cluster_id** in table **kws**.

In [ ]:
df_reviews = pd.read_sql_query("""SELECT review_id, review, game_id 
                    FROM reviews JOIN games USING(game_id);""", conn)

In [ ]:
def get_ngram(x, ngram, min_df=1):
    vec = CountVectorizer(ngram_range=[ngram, ngram], min_df=min_df).fit(x)
    bow = vec.transform(x)
    sum_words = bow.sum(axis = 0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse = True)
    return words_freq

def bigram_rules(bigram): 
    first_pos = set(['ADJ', 'NOUN'])
    second_pos = set(['NOUN'])
    
    tags = [token.pos_ for token in nlp(bigram, disable=['parser','ner'])]
    
    return tags[0] in first_pos and tags[1] in second_pos

In [ ]:
def unigram_rules(unigram): 
    first_pos = set(['NOUN', 'PROPN'])
    tags = [token.pos_ for token in nlp(unigram, disable=['parser', 'ner'])]
    return tags[0] in first_pos

In [ ]:
unigram_freq = get_ngram(sents_prep, 1, 3)
unigram_df = pd.DataFrame(unigram_freq, columns=['unigram', 'freq'])

In [ ]:
bigram_vocab = ' '.join(bigram_df['bigram'])

In [ ]:
unigram_df_50 = []
count = 0
for unigram, freq in zip(unigram_df['unigram'], unigram_df['freq']): 
    if unigram_rules(unigram) and unigram not in bigram_vocab:
        unigram_df_50.append((unigram, freq))

        count += 1

        if count == 50: 
            break

In [521]:
df_sent_prep = pd.read_sql_query("""SELECT sent, sent_id, game_id
                    FROM sents JOIN reviews USING(review_id) JOIN games USING(game_id);""", conn)

In [522]:
df_sents = df_sent_prep[df_sent_prep['game_id'] == 1][['sent', 'sent_id']]

In [ ]:
for sent in sents_10: 
    print(f'> {sent}')

In [23]:
def get_tokens_index_dict(string):
    index_dict = {}
    index = 0 
    for i, char in enumerate(string): 
        if char == ' ':
            index += 1
        else: 
            index_dict[i] = index
    
    return index_dict

In [558]:
def get_stopwords(word_list=[]):
    stopwords = nlp.Defaults.stop_words.copy()  
    stopwords |= set(word_list)
    return stopwords

# remove stopwords before grouping
stop_words = get_stopwords(['game', 'games', 'lot', 'lots', 'ton', 'tons', 'bit', 'bits', 'fun', 
                            'way', 'ways', 'thing', 'things', 'time', 'times', 'type', 'types', 
                               'opinion', 'opinions', 'sense', 'terms', 'lack', 'fact'])

In [553]:
# check if a keyword is messy
def is_messy(kw): 
    return re.search('[^a-zA-Z0-9\s\-:,;]', kw)

def clean_token(kw_token):
    kw_token = kw_token.lower()
    kw_token = re.sub('[-:]', '', kw_token)
    return kw_token

In [554]:
def get_kws_sent_ids(df_sents):
    kws_sent_ids = set()

    noun_types = set(['NOUN', 'PROPN'])
    adj_types = set(['ADJ'])
    pattern = re.compile('(ADJ (PUNCT ADJ )*)*((NOUN|PROPN) (PUNCT (NOUN|PROPN) )*(PUNCT )?)*(NOUN|PROPN)')

    for sent, sent_id in zip(df_sents['sent'], df_sents['sent_id']): 
        doc = nlp(sent, disable=['parser', 'ner'])

        tokens = [token.text for token in doc]
        tags_str = ' '.join([token.pos_ for token in doc])
        tokens_index_dict = get_tokens_index_dict(tags_str)
        matches = pattern.finditer(tags_str)
        
        for match in matches: 
            tags = [elem for elem in match.group(0).split()]
            
            kw = [] 
            index = match.start()
            
            for i, tag in enumerate(tags): 
                kw_token = clean_token(tokens[tokens_index_dict[index]])
                index += len(tag) + 1
                
                if kw_token in (',', ';') and tags[i - 1] in ('PROPN', 'NOUN') and len(kw) != 0:
                    kw = ' '.join(kw)

                    if not is_messy(kw):
                        kws_sent_ids.add((kw, sent_id))

                    kw = []
                
                # remove all adjective before a stop word, otherwise would end up with lots of ADJ alone keywords 
                elif kw_token in stop_words and tag in ('PROPN', 'NOUN') and tags[i - 1] in ('ADJ'): 
                    kw = []
                    
                elif kw_token not in (',', ';') and kw_token not in stop_words and len(kw_token) != 0:
                    kw.append(kw_token)

                    
            if len(kw) != 0:
                kw = ' '.join(kw)

                if not is_messy(kw):
                    kws_sent_ids.add((kw, sent_id))
    
    return kws_sent_ids

In [555]:
def get_kws_sent_ids_group(kws_sent_ids):
    res={}
    
    for kw_sent_id in kws_sent_ids:
        if kw_sent_id[0] not in res:
            res[kw_sent_id[0]] = [kw_sent_id[1]]
        else:
            res[kw_sent_id[0]].append(kw_sent_id[1])
            
    return res

In [556]:
kws_sent_ids = get_kws_sent_ids(df_sents[:1000])
kws_sent_ids_group = get_kws_sent_ids_group(kws_sent_ids)

In [343]:
# 1000 sents > 1562 
# 100 sents > 243 

In [559]:
pprint(kws_sent_ids_group)

{'2d': [4718],
 '2d dark souls': [4724, 3720, 1967, 2384, 4948],
 '2d dark souls vibe': [1611],
 '2d feel': [5219],
 '2d metroidvania': [3566],
 '2d pixelated art style': [2707],
 '2d sidescrolling metroidvania style': [4839],
 '2nd try': [1091],
 'aaa': [1827],
 'abilities': [517, 399, 2767, 3063],
 'ability': [1934, 1754, 314, 3115, 1759, 5236],
 'access': [345],
 'accounts': [4280],
 'accuracy': [6270],
 'achievement': [6069, 5173],
 'achievementhunting': [3896],
 'achievements': [2183, 534, 3920, 5623, 3830],
 'achivements': [2549],
 'action': [4491],
 'action adventure': [5538],
 'action platform': [2913],
 'action platformer': [1800, 1905, 5190, 1969],
 'action platformer itch': [3128],
 'action platformers': [2673, 3955],
 'active item slot': [3481],
 'active item slots': [3481, 3480, 4460],
 'active items': [5710],
 'actives': [4460],
 'actual achievements': [2440],
 'actual metroid': [4807],
 'actual shards': [1558],
 'addition': [804],
 'additional challanges': [3830],
 'ador

 'fate': [5550, 2758],
 'favorites': [423],
 'favourite boss fights': [2658],
 'favourite elements': [1375],
 'feature': [5272],
 'females': [1997],
 'fennel': [5580],
 'fidget animations': [3333],
 'fight': [1384, 4461, 5648, 5943, 6376, 4431],
 'fighting': [4587, 3312],
 'fighting bosses': [3491],
 'fights': [4465],
 'filthy casuals': [3597],
 'final boss': [5696,
                1764,
                5242,
                4486,
                1091,
                3583,
                4859,
                3660,
                2199,
                5580,
                4720,
                5695,
                3652,
                2278,
                5116,
                821,
                3578,
                1546,
                4145,
                490,
                1470,
                3738,
                801,
                5549,
                3021,
                3654,
                5875,
                5550,
                1553,
                36

 'publisher': [3024],
 'punishing dark souls feel': [2758],
 'purpose': [5701],
 'puzzles': [5370, 1375],
 'quality': [2946, 4590, 1037],
 'quality pixel art animations': [2874],
 'quarrel': [3876],
 'questionable design choices': [2545],
 'questions': [1670],
 'quick experience': [2168],
 'rabi ribi': [33],
 'radar': [2622, 6349],
 'rage': [5786],
 'random spot': [1384],
 'randomness': [2757],
 'range': [3456],
 'range attacks': [2760],
 'ranged attacks': [1934],
 'rdein': [5239],
 'reaching new heights': [4491],
 'reactive controls': [3144],
 'real aerial combos': [1552],
 'real letdown': [3738],
 'real reason': [5116],
 'real treat': [2102],
 'real upgrades': [341],
 'reason': [5200, 1408, 1048],
 'reasonable price': [3691],
 'redeeming aspect': [3327],
 'regards': [3480, 3468],
 'regrets': [4932],
 'repeat playthroughs': [5623],
 'replay value': [2102,
                  6069,
                  4490,
                  5301,
                  4482,
                  5620,
           

In [560]:
kws = list(kws_sent_ids_group.keys())

In [561]:
kw_embeddings = embedder.encode(kws)

# Normalize the embeddings to unit length
kw_embeddings = kw_embeddings /  np.linalg.norm(kw_embeddings, axis=1, keepdims=True)

In [562]:
# perform agglomerative clustering
clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=0.6)
clustering_model.fit(kw_embeddings)
cluster_assignment = clustering_model.labels_

In [563]:
kw_clusters = {}
for kw_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in kw_clusters:
        kw_clusters[cluster_id] = []

    kw = kws[kw_id]
    kw_clusters[cluster_id].append((kw, kws_sent_ids_group[kw]))

In [564]:
print(len(kw_clusters))

306


In [ ]:
# with conn:
#     cursor.executemany("""INSERT INTO clusters (game_id, cluster_num, cluster_name) VALUES 
#                         (?, ?, ?);""", cluster_tuples)    
    
# with conn:
#     cursor.executemany("""INSERT INTO kws (kw, freq, cluster_id) VALUES 
#                     (?, ?, (SELECT cluster_id FROM clusters WHERE cluster_num=? AND game_id=?));""",
#                    kw_tuples)

# with conn:
#     cursor.executemany("""INSERT INTO clusters_sents  (cluster_id, sent_id) VALUES 
#                         (?, ?);""", cluster_sent_tuples)

In [431]:
# pprint(kw_clusters)

In [565]:
kw_clusters

{170: [('music',
   [3714,
    2384,
    5239,
    1205,
    5797,
    6101,
    5483,
    2398,
    2645,
    3846,
    3822,
    532,
    2531,
    2567,
    550,
    2584,
    3870,
    6195,
    585,
    4754,
    2953,
    415,
    6260,
    2565,
    2346,
    990,
    4940,
    1018,
    1082,
    2126,
    2022,
    2658,
    1291,
    1771,
    4564,
    3557,
    1680,
    4098,
    6342,
    202,
    506]),
  ('sound design',
   [4477, 5344, 2126, 5050, 1772, 2693, 585, 2398, 3095, 3822]),
  ('special sound effect', [5943]),
  ('great music',
   [3005, 3925, 4826, 3128, 5306, 5215, 572, 5873, 4213, 3169, 41]),
  ('extra sounds', [5963]),
  ('pretty sound track', [6367]),
  ('sound', [2167, 3105, 2713]),
  ('sound effects', [646, 2713, 415, 48, 2531, 2224, 746, 5963, 4754, 2323]),
  ('great soundtrack',
   [3889, 2514, 5358, 423, 2736, 682, 5457, 5837, 4055, 1771, 5372, 1160]),
  ('soundtrack', [5194, 746, 3137, 5542]),
  ('similar sound', [4452]),
  ('resonant music', [152])

In [600]:
# get the list of biggest clusters 
def get_biggest_clusters(kw_clusters, top_n=50):
    cluster_nums_sizes = []

    for cluster_num, kws_sents in kw_clusters.items():
        sents_num = sum([len(kw_sents[1]) for kw_sents in kws_sents])
        cluster_nums_sizes.append((cluster_num, sents_num))

    cluster_nums_sizes_sorted = sorted(cluster_nums_sizes, key=lambda x: x[1], reverse=True)
    return set([cluster_num_size[0] for cluster_num_size in cluster_nums_sizes_sorted[:top_n]])

In [603]:
biggest_clusters = get_biggest_clusters(kw_clusters)

In [604]:
game_id = 1

cluster_tuples = []
kw_tuples = []
clusters_sents = []

for cluster_num, kws_sent_ids in kw_clusters.items(): 
    cluster_num = cluster_num
    game_id = game_id
    
    if cluster_num in biggest_clusters:
        kws_sent_ids_sorted = sorted(kws_sent_ids, key=lambda x: len(x[1]), reverse=True)
        cluster_name = kws_sent_ids_sorted[0][0]
        cluster_tuples.append((game_id, cluster_num, cluster_name))

        sents_num = sum([len(kw_sent_ids[1]) for kw_sent_ids in kws_sent_ids])

        for kw_sent_ids in kws_sent_ids: 
            kw = kw_sent_ids[0]
            sent_ids = kw_sent_ids[1]
            freq = len(kw_sent_ids[1])
            kw_tuples.append((kw, freq, cluster_num, game_id))

            for sent_id in sent_ids: 
                clusters_sents.append((sent_id, cluster_num, game_id))

In [611]:
print(cluster_tuples[0])
print(kw_tuples[0])
print(clusters_sents[0])

(1, 170, 'music')
('music', 41, 170, 1)
(3714, 170, 1)


In [474]:
print(cluster_tuples[0])
print(kw_tuples[0])
print(clusters_sents[0])

(1, 130, 'music')
('music', 41, 130, 1)
(3714, 130, 1)


In [ ]:
with conn:
    cursor.executemany("""INSERT INTO clusters (game_id, cluster_num, cluster_name) VALUES 
                        (?, ?, ?);""", cluster_tuples)    
    
with conn:
    cursor.executemany("""INSERT INTO kws (kw, freq, cluster_id) VALUES 
                    (?, ?, (SELECT cluster_id FROM clusters WHERE cluster_num=? AND game_id=?));""",
                   kw_tuples)

with conn:
    cursor.executemany("""INSERT INTO clusters_sents  (sent_id, cluster_id) VALUES 
                        (?, (SELECT cluster_id FROM clusters WHERE cluster_num=? AND game_id=?));""", cluster_sent_tuples)

In [ ]:
# cluster_tuples = []
# kw_tuples = []

# for game_id in df_sent_prep['game_id'].unique():
#     sents_prep = df_sent_prep[df_sent_prep['game_id'] == game_id]['sent_prep']
    
#     bigram_freq = get_ngram(sents_prep, 2, 3)
#     bigram_df = pd.DataFrame(bigram_freq, columns=['bigram', 'freq'])
    
#     bigram_df_50 = []
#     count = 0
    
#     for bigram, freq in zip(bigram_df['bigram'], bigram_df['freq']): 
#         if bigram_rules(bigram):
#             bigram_df_50.append((bigram, freq))

#             count += 1

#             if count == 50: 
#                 break
                
#     bigram_df = pd.DataFrame(bigram_df_50, columns=['bigram', 'freq'])
    
#     kws = bigram_df['bigram']
#     kw_embeddings = embedder.encode(kws)
    
#     # Normalize the embeddings to unit length
#     kw_embeddings = kw_embeddings /  np.linalg.norm(kw_embeddings, axis=1, keepdims=True)
    
#     # perform agglomerative clustering
#     clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=0.6)
#     clustering_model.fit(kw_embeddings)
#     cluster_assignment = clustering_model.labels_
    
#     kw_clusters = {}
#     for kw_id, cluster_id in enumerate(cluster_assignment):
#         if cluster_id not in kw_clusters:
#             kw_clusters[cluster_id] = []

#         kw_clusters[cluster_id].append(kws[kw_id])
        
#     bigram_df['cluster_num'] = cluster_assignment
    
#     for cluster_num, cluster_val in kw_clusters.items():
#         cluster_tuples.append((int(game_id), int(cluster_num), cluster_val[0]))
    
#     for kw, freq, cluster_num in zip(bigram_df['bigram'], bigram_df['freq'], bigram_df['cluster_num']):
#         kw_tuples.append((kw, freq, int(cluster_num), int(game_id)))

In [ ]:
# with conn:
#     cursor.executemany("""INSERT INTO clusters (game_id, cluster_num, cluster_name) VALUES 
#                         (?, ?, ?);""", cluster_tuples)    
    
# with conn:
#     cursor.executemany("""INSERT INTO kws (kw, freq, cluster_id) VALUES 
#                     (?, ?, (SELECT cluster_id FROM clusters WHERE cluster_num=? AND game_id=?));""",
#                    kw_tuples)

### Step 4: 
- Loop through **sent_prep** in table **sents**, fuzzy-match each **kw** in table **kws**. 
    - Insert **cluster_id**, **sent_id** in table **clusters_sents** to link table **clusters** and **sents**.

In [ ]:
# df_sent_prep = pd.read_sql_query("""SELECT game_id, sent_prep, sent_id
#                     FROM sents JOIN reviews USING(review_id) JOIN games USING(game_id);""", conn)

# df_kw = pd.read_sql_query("""SELECT game_id, cluster_id, kw 
#                     FROM kws LEFT JOIN clusters USING(cluster_id);""", conn)

In [ ]:
# cluster_sent_tuples = []

# for game_id in df_sent_prep['game_id'].unique():
#     sents_prep = df_sent_prep[df_sent_prep['game_id'] == game_id]
#     kws = df_kw[df_kw['game_id'] == game_id]
    
#     kw_clusters = {}
#     for kw, cluster_id in zip(kws['kw'], kws['cluster_id']):
#         if cluster_id not in kw_clusters:
#             kw_clusters[cluster_id] = []

#         kw_clusters[cluster_id].append(kw)
        
#     for sent_id, sent_prep in zip(sents_prep['sent_id'], sents_prep['sent_prep']):
#         for cluster_id, kws in kw_clusters.items():
#             for kw in kws:
#                 matches = find_near_matches(kw, sent_prep, max_l_dist=1)

#                 if len(matches) != 0: 
#                     cluster_sent_tuples.append((cluster_id, sent_id))
#                     break

In [ ]:
# with conn:
#     cursor.executemany("""INSERT INTO clusters_sents  (cluster_id, sent_id) VALUES 
#                         (?, ?);""", cluster_sent_tuples)

### Step 5:
- Remove all **sent_id** in table **sents** if they don't exist in table **clusters_sents**.
    - Insert **score_flair**, **score_vader**, **recommended**, **score_total**, **sent_embedding** in table **sents**.

In [ ]:
# remove all sentences that contains no keyword from table sents
with conn: 
    cursor.execute("""
        DELETE FROM sents 
        WHERE sent_id NOT IN (
            SELECT DISTINCT sent_id
            FROM clusters_sents);
    """)

In [ ]:
df_sent = pd.read_sql_query("""SELECT sent_id, sent, recommended
                                FROM sents JOIN reviews USING(review_id);""", conn)

In [ ]:
df_sent

In [ ]:
def get_score_flair(sent, threshold=0.9): 
    sent_flair = flair.data.Sentence(sent)
    sentiment_model_fast.predict(sent_flair)
    
    value_flair = sent_flair.labels[0].value
    score_flair = sent_flair.labels[0].score
    
    value_flair = 1 if value_flair == 'POSITIVE' else -1
    
    score_flair = value_flair*score_flair
    
    return 1 if score_flair > threshold else -1 if score_flair < -threshold else 0

In [ ]:
sentis = df_sent['sent'].map(get_score_flair) 

In [ ]:
sent_embeddings = embedder.encode(df_sent['sent'])

sent_embeddings = sent_embeddings /  np.linalg.norm(sent_embeddings, axis=1, keepdims=True)

sent_embeddings_str = [pickle.dumps(sent_embedding) for sent_embedding in sent_embeddings]

In [ ]:
sent_tuples = []

for senti, sent_embedding_str, sent_id in zip(sentis, sent_embeddings_str, df_sent['sent_id']): 
    sent_tuples.append((senti, sent_embedding_str, sent_id))     

In [ ]:
with conn:
    cursor.executemany("""UPDATE sents SET (senti, sent_embedding) = (?, ?)
                        WHERE sent_id=?;""", sent_tuples)

In [ ]:
df_sent_embedding = pd.read_sql_query("""SELECT sent_embedding FROM sents;""", conn)
sent_embeddings = np.array([pickle.loads(sent_embedding) for sent_embedding in df_sent_embedding['sent_embedding']])

In [475]:
df_sent = pd.read_sql_query("""SELECT cluster_id, sent_id, sent, sent_embedding, senti, cluster_name, game_id
                                FROM clusters_sents LEFT JOIN(sents) USING(sent_id)
                                                    LEFT JOIN(clusters) USING(cluster_id);""", conn)

In [476]:
def decode_sent_embeddings(sent_embeddings):
    return np.array([pickle.loads(sent_embedding) for sent_embedding in sent_embeddings])

In [ ]:
game_id = 1

In [477]:
df_sent = df_sent[df_sent['game_id'] == game_id]

In [479]:
df_sent

,cluster_id,sent_id,sent,sent_embedding,senti,cluster_name,game_id
0,1,1,Metroidvania with some influences from Dark So...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,0,dark soul,1
1,11,3,I had to check some guide to see the true ending.,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,0,true ending,1
2,16,21,A short but sweet metroidvania that is worth y...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,1,worth price,1
3,9,25,It's a 2-d action platformer with a simple yet...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,1,action platformer,1
4,20,25,It's a 2-d action platformer with a simple yet...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,1,range attack,1
...,...,...,...,...,...,...,...
4188,2,22073,From a metroidvania perspective the gameplay h...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,-1,boss fight,1
4189,3,22080,"Con un diseo ""pixel-art"", escenarios en 2D, y ...",b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,1,pixel art,1
4190,1,22085,One of the first wave of dark souls inspired m...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,1,dark soul,1
4191,6,22086,I rate it much higher than Hollow Knight.,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...,0,hollow knight,1


In [478]:
df_sent_1 = df_sent[['senti', 'cluster_name']].groupby(['cluster_name', 'senti']).size().reset_index(name='count')

df_sent_1['sum_count'] = df_sent_1['count'].groupby(df_sent_1['cluster_name']).transform('sum')

df_sent_1 = df_sent_1.sort_values(by=['sum_count', 'senti'], ascending=[False, True])

df_sent_1

,cluster_name,senti,count,sum_count
6,dark soul,-1,77,675
7,dark soul,0,299,675
8,dark soul,1,299,675
51,pixel art,-1,22,606
52,pixel art,0,64,606
...,...,...,...,...
38,long time,1,24,40
41,main character,1,19,40
48,passive item,-1,10,39
49,passive item,0,19,39


In [ ]:
colors_dict  = {
    1:'#26de81',
    0:'#fed330',
    -1:'#fc5c65'
}

colors = [colors_dict[k] for k in df_sent_1['senti'].values]

# fig = go.Figure(data=[
#     go.Bar(x=df_sent_1['cluster_name'], 
#            y=df_sent_1['count'],
#           marker_color=colors)
# ])
fig = go.Figure()
fig.add_trace(go.Bar(name="positive", x=df_sent_1[df_sent_1['senti'] == 1]['cluster_name'], y=df_sent_1[df_sent_1['senti'] == 1]['count'], marker_color='#26de81'))
fig.add_trace(go.Bar(name="neutral", x=df_sent_1[df_sent_1['senti'] == 0]['cluster_name'], y=df_sent_1[df_sent_1['senti'] == 0]['count'], marker_color='#fed330'))
fig.add_trace(go.Bar(name="negative", x=df_sent_1[df_sent_1['senti'] == -1]['cluster_name'], y=df_sent_1[df_sent_1['senti'] == -1]['count'], marker_color='#fc5c65'))

# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

In [480]:
df_sent_cluster = df_sent[(df_sent['cluster_name'] == 'dark soul') & (df_sent['senti'] == -1)]

In [ ]:
for sent in df_sent_cluster['sent']: 
    print(f'>>> {sent}')

In [481]:
sents = df_sent_cluster['sent'].reset_index(drop = True)
sent_embeddings = decode_sent_embeddings(df_sent_cluster['sent_embedding'])

In [482]:
sents

0     The Bad: low monster variety, combat is simpli...
1     The difficulty is a bit extreme for me, the co...
2                             It's not Dark Souls hard.
3     Difficulty wise, I'd put it about the same as ...
4     The music and the setting try to be Dark Souls...
                            ...                        
72    All of these things exist outside of Dark Soul...
73    This game feels like the lovechild of Dark Sou...
74        Not like Dark Souls (someone else said that).
75    1 and 2 were Cave Story-esque without the focu...
76    I never really got into Dark Souls, and I thin...
Name: sent, Length: 77, dtype: object

In [ ]:
# # PCA with 3 dimensions
# pca = PCA(n_components=3)
# components = pca.fit_transform(sent_embeddings)

# total_var = pca.explained_variance_ratio_.sum() * 100

In [ ]:
# fig = px.scatter_3d(
#     components, x=0, y=1, z=2, color=clustering_model.labels_.astype(np.str),
#     title=f'Total Explained Variance: {total_var:.2f}%',
#     labels={'0': 'PC 1', '1': 'PC 2', '2': 'PC 3'}, 
#     hover_data=[sents],
# )
# fig.show()

In [485]:
df_sent = pd.read_sql_query("""SELECT sent, sent_embedding
                                FROM sents JOIN reviews USING(review_id)
                                WHERE game_id=1;""", conn)

In [486]:
df_sent

,sent,sent_embedding
0,Metroidvania with some influences from Dark So...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
1,I had to check some guide to see the true ending.,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
2,A short but sweet metroidvania that is worth y...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
3,It's a 2-d action platformer with a simple yet...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
4,"Other than that, there isn't a ton of replay v...",b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
...,...,...
3769,From a metroidvania perspective the gameplay h...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
3770,"Con un diseo ""pixel-art"", escenarios en 2D, y ...",b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
3771,One of the first wave of dark souls inspired m...,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...
3772,I rate it much higher than Hollow Knight.,b'\x80\x03cnumpy.core.multiarray\n_reconstruct...


In [487]:
sents = df_sent['sent'].reset_index(drop = True)
sent_embeddings = decode_sent_embeddings(df_sent['sent_embedding'])
n_sents = 5

In [490]:
def get_centroid(arr):
    length, dim = arr.shape
    return np.array([np.sum(arr[:, i])/length for i in range(dim)])

# return index of the vectors in corpus_embeddings nearest to the centroid
def get_nearest_indexes(centroids, corpus_embeddings):
    return vq(centroids, corpus_embeddings)[0]

In [503]:
def hdbscan_sum(sents, sent_embeddings, n_sents):
    # Perform hdbscan clustering
    clustering_model = hdbscan.HDBSCAN(min_cluster_size=2)
    clustering_model.fit(sent_embeddings)
    cluster_assignment = clustering_model.labels_

    clustered_sents = {}
    clustered_sent_embeddings = {}

    for sent_id, cluster_id in enumerate(cluster_assignment):
        if cluster_id not in clustered_sents:
            clustered_sents[cluster_id] = []
            clustered_sent_embeddings[cluster_id] = []

        clustered_sents[cluster_id].append(sents[sent_id])
        clustered_sent_embeddings[cluster_id].append(sent_embeddings[sent_id])
    
    pprint(clustered_sents)

    unique, counts = np.unique(cluster_assignment, return_counts=True)

    unique_counts = list(zip(unique, counts))

    unique_counts = sorted(unique_counts, key=lambda x: x[1], reverse=True)

    sents_count = 0 

    sents_sum = []

    for unique_count in unique_counts:
        cluster_num = unique_count[0]

        if cluster_num != -1:
            centroid = get_centroid(np.array(clustered_sent_embeddings[cluster_num]))
            centroid = np.array([centroid])
            
            index = get_nearest_indexes(centroid, clustered_sent_embeddings[cluster_num])[0]
            sents_sum.append(clustered_sents[cluster_num][index])

            sents_count += 1
            if sents_count == n_sents: 
                break
    
    return sents_sum

In [504]:
hdbscan_sum(sents,  sent_embeddings, 10)

{-1: ['A short but sweet metroidvania that is worth your time, especially if '
      'you like getting difficult achievements!',
      "It's a 2-d action platformer with a simple yet rewarding combat system "
      'and difficulty options aplenty.',
      "If you like games like Rabi-Ribi or Metroidvania games in general you'd "
      'def like this game!!',
      'A charming and difficult pixel action platformer.',
      'Worth the pain of repeated deaths for a perfect boss fight.',
      'Still Momodora: Reverie Under the Moonlight is a must for newcomers or '
      'veterans of the metroidvania genre looking for something new, charming '
      'and short.',
      'The boss fights are hands down the most fun and challenging part of '
      'this game.',
      'If you are scrambling to binge another metroidvania, than Momodora '
      'Reverie of the Moonlight is a fun one worthy of your time.',
      'Decent Metroidvania, Slightly Challenging, Janky Hitboxes on Higher '
      "Diffic

      'decided to respond.',
      'There are no extra modes aside from higher difficulties.',
      "I had no problems beating the 1 HP mode on my second run and I'm a "
      "mediocre platformer player (still can't beat the true ending dungeon in "
      'Cave Story).',
      'Great soundtrack and lovely level design.',
      'I highly advice to buy this game on sales but if you want to support '
      'the dev team go ahead and buy it on full price.',
      'I love the art and animation.',
      'The music is creepy and depressing in a good way, and the sound design '
      'is fantastic.',
      "A well-made metroidvania that, despite it's cute art-style, tells a "
      'story of sacrifice and death.',
      'Beautiful art-style with solid pixel art.',
      'The boss was easier when I gave up on trying to strategize and just '
      'went up and whaled on it until the boss fight was over.',
      'The pixel art is very well drawn, and the landscapes are gorgeous.',
      'Conclu

      'Insanely fun to play & crisp controls.',
      'Momodora 4 sports what I consider at the moment the golden standard of '
      'difficulty curves.',
      "It's a short game, but with a good gameplay and a really charming pixel "
      'art.',
      "It's the Dark Souls of video games.",
      'It basically takes the same points that most people like about Dark '
      'Souls, condensed into a short, tight package.',
      'A very engaging adventure game with great pixelart and a unique story.',
      'I could talk about the nicely tuned difficulty settings and the tight '
      'controls which defintitely wont fail you, or the dark soul like story '
      'telling, where only pieces are laid before you, and it is up to you to '
      'to put the pieces of the story and the characters together to form your '
      'own picture.',
      'It has gorgeous gothic themed pixel art, fluid controls and great '
      'soundtrack.',
      'HAVE YOU EVER WANTED DARK SOULS TO BE HARDER AND

      'travelling to various locations in an attempt to take down the '
      "corrupted Queen who has infected Kaho's village of Lan, in all "
      'seriousness, this is a really good game, as someone who plays 2D '
      'Side-Scrolling Platformers often, I highly reccomend this game to '
      'anyone who is also a fan, the game itself is fairly short, but I find '
      "it has good replay value, I've found myself playing through this game "
      'multiple times already and I am currently speedrunning it and planning '
      'to pick up 100% for it fairly soon, once you actually find where '
      "everything is, it's hard to forget, although I still find myself making "
      "mistakes over and over again and if you're looking for a challenge, I "
      'reccomend trying to beat Hard Difficulty and even Insane Difficulty '
      'which you unlock for beating Hard, I am yet to beat Insane Difficulty '
      'but I wish any who try the best of luck.',
      "I'm a huge castlevania

 180: ["If you're a fan of exploration-based action platformers (like Cave "
       "Story), then I'd say that this game is an essential.",
       'As it is, though, I would still highly recommend the game to anyone '
       'that enjoys action-platformers or good games in general.',
       "I'd absolutely recommend this game to any fans of action platformers "
       'that are alright with what might be a fairly low playtime.',
       'I highly recommend this series to anyone looking for an action '
       'platformer to play.'],
 181: ['The high difficulty and incentive to keep going (in example, each boss '
       'will give you a special drop if you defeat them without taking any '
       'damage yourself), plentiful enemy respawns, and even a couple of enemy '
       "ambushes here and there resembled the Souls' games distinctive style- "
       'just transferred to a 2D world.',
       'Good bosses and sadistically placed enemies who will frequently 1-2 '
       'shot you on Hard

["It's worth the price.",
 'Great soundtrack.',
 'Also, you can kill a Witch by slapping her in the boobs with a giant maple leaf.',
 'Metroidvania with some influences from Dark Souls.',
 'Great metroidvania.',
 "One of the best metroidvania games I've played.",
 'A fantastic, very well made action platformer with great tone and engaging gameplay.',
 'Good replay value.',
 'Fun boss fights.',
 'Hollow Knight.']

In [493]:
def kmeans_sum(sents,  sent_embeddings, n_sents=5):
    sents_sum = []

    clustering_model = KMeans(n_clusters=n_sents, random_state=0)
    clustering_model.fit(sent_embeddings)

    cluster_assignment = clustering_model.labels_
    centroids = clustering_model.cluster_centers_

    clustered_sentences = [[] for i in range(n_sents)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(sents[sentence_id])

    indexes = get_nearest_indexes(centroids, sent_embeddings)
    
#     closest, distances = vq(centroids, sent_embeddings_pca)

    for index in indexes: 
        sents_sum.append(sents[index])
        
    return sents_sum

In [494]:
kmeans_sum(sents, sent_embeddings, 10)

['Its a little short (3-5 hours on your first playthrough), but it has different endings and a decent replay value, overall its really fun.',
 'Has some challenging boss fights.',
 'The art style is amazing, and along with the soundtrack it creates a unique atmosphere.',
 'Speaking of that the game is also fairly inspired by Dark Souls.',
 'Very good metroidvania styled game.',
 'Solid game, worth the price.',
 'So you have a melee attack and a ranged attack.',
 'Awesome game, this game has great controls and the artstyle is quite pretty.',
 'Fantastic pixel art.',
 "Hard mode is actually fairly challenging as well and I've been having fun playing though it."]

In [507]:
def agglo_sum(sents, sent_embeddings, n_sents=5, threshold=0.5): 
    # Perform agglomerative clustering
    clustering_model = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=threshold)
    clustering_model.fit(sent_embeddings)
    cluster_assignment = clustering_model.labels_

    clustered_sents = {}
    clustered_sent_embeddings = {}

    for sent_id, cluster_id in enumerate(cluster_assignment):
        if cluster_id not in clustered_sents:
            clustered_sents[cluster_id] = []
            clustered_sent_embeddings[cluster_id] = []

        clustered_sents[cluster_id].append(sents[sent_id])
        clustered_sent_embeddings[cluster_id].append(sent_embeddings[sent_id])

    pprint(clustered_sents)    
        
    unique, counts = np.unique(cluster_assignment, return_counts=True)

    unique_counts = list(zip(unique, counts))

    unique_counts = sorted(unique_counts, key=lambda x: x[1], reverse=True)

    sents_count = 0 

    sents_sum = []

    for unique_count in unique_counts:
        cluster_num = unique_count[0]

        centroid = get_centroid(np.array(clustered_sent_embeddings[cluster_num]))
        centroid = np.array([centroid])

        index = get_nearest_indexes(centroid, clustered_sent_embeddings[cluster_num])[0]
        sents_sum.append(clustered_sents[cluster_num][index])

        sents_count += 1
        if sents_count == n_sents: 
            break
    
    return sents_sum

In [508]:
agglo_sum(sents, sent_embeddings, 10)

{0: ['Easily worth full price.',
     "It's worth the price.",
     'Well worth the money.',
     'It is 100% worth your money.',
     "But it's still a good time and worth the pricetag.",
     'Absolutely worth your time and money!',
     "Overall, it's worth the money.",
     "It's worth your time and money.",
     'It is totally worth its full price even when not on sale.',
     'More then worth its price.',
     "I could say it'd be worth fully price.",
     "It's worth the full price imo really.",
     '100% worth your money at any price.',
     'Not worth the full price but it was a lot of fun, so buy it on sale :)',
     'Easily worth the price of admission.',
     "It's definitely well worth the price.",
     "It totally worth its money, don't worry.",
     'Definitely worth the price of admission.',
     "Most definitely worth the money, I'd even pay twice the full price for "
     'it.',
     'Awesome, definitely worth your time.',
     'Pretty good buy nonetheless and worth 

      'till now?',
      'Interesting boss fight.',
      'Story not interesting neither are the characters(personal opinion) - '
      'one frustrating boss fight that is based on randomness(easy but takes '
      'forever or it can be quick depending how lucky you get) -Most boss '
      'fight were easy especially when you understand the pattern(personal '
      'opinion)',
      'The boss fights are very hard at first, but very fair once you start to '
      'learn them.',
      'With tight controls heavy emphasis on exploration and intense boss '
      "fights, This is a game that tends to take it's influences and creates a "
      'compelling game that once you get going, is very hard to put down.',
      'Pretty much all bosses have random attack orders and this could end up '
      'in your "favor", allowing you to just spam your ranged attack to kill '
      'them... or it can end up just requiring you to run around for 80% of '
      'the fight since the enemy is offscreen th

      'heart.',
      'It made me learn from my mistakes, and by the end of the game I was far '
      'more confident when tackling a tough enemy or boss.',
      'However, when I reached the third boss I fought I was astounded.',
      'I never felt like I was fighting the controls, and knew when I died it '
      'was my own fault, which provided a good challenge and great '
      'satisfaction when you polished off another boss.',
      'I found the platforming to be fun and engaging and the boss fights were '
      'very well designed with a few examples of genius.',
      'Winning boss fights felt like a challenge and were very rewarding to '
      'actually finish.',
      'The bossfights are very difficult for me, but they were so fun to '
      'experience.',
      'I enjoyed exploring, but found it far too easy to wander into the next '
      'boss battle without completing an area, with little incentive to go '
      'back rather than continuing the story (boss battles shoul

      'Imagine Castlevania with Darksouls mechanics sprinkled in.',
      'A must play if you are a fan of the Castlevania Series and to a lesser '
      'extent Dark Souls, it may be a short game.',
      'A beautiful, atmospheric, sometimes challenging and always enjoyable '
      'indie platformer that takes a big drop of Castlevania SOTN, a pinch of '
      'Dark Souls and mixes it up into something a bit magical.',
      'I wont write too much but basically this game is what you get if you '
      'mixed dark souls with castlevania in 2d ! and leaning a bit more '
      'towards castlevania.',
      'Great mix between castlevania and dark souls.',
      'For 7$ this game is amazing; it has a little of megaman zero, dark '
      'souls, and castlevania in it.',
      'The classic style of 2D platformers like Castlevania and Metriod, with '
      'some modern inspiration from games like Dark Souls.',
      'Protagonist can transform into a cat; + So many things remind me of '
      

       "or going for all the achievements, I don't see too many people coming "
       'back to the game.'],
 208: ['The dark theme looks awesome, great boss fight along with nice '
       'soundtrack, my complain would be that it is too short and the bosses '
       'are not very challenging even in hard mode hit run.',
       'Sometimes (on some boss fights) soundtrack is "not enough emotional" '
       'to the situation. -',
       'The world music feels like world music and the boss music is certainly '
       'done up to make it really feel like a boss fight.',
       'The soundtrack is good,mostly helping to strengthen the atmosphere, '
       'but also pumping up on certain boss fights to ringle your tingle.',
       'I rarely find myself enjoying the stage music almost as much as the '
       'boss battle.',
       "It's a dark and depressing world, and the themes I've encountered "
       'thusfar are good, while the boss battles are good for getting you '
       'psyched and 

 415: ['That said, the game is a little bit shorter than what I would like, '
       'but you can spend many hours with Kaho exploring the city of Karst.'],
 416: ["The difficulty is artificial, it's the result of the bad control "
       'mechanics and lack of flexibility rather than clever design.',
       "The design isn't quite tight enough to make insane difficulty really "
       'work well, IMO.'],
 417: ["In hard everything kills you in one or two hit, which i wouldn't mind "
       'if the hitboxes and dodge roll made sense.',
       'Three hits from most monsters will kill you, with your only mercies '
       'being a (rather weak) projectile and a dodge-roll with limited '
       'I-frames.',
       'You WILL die to many of them your first time, but proper mastery of '
       'ranged combat, healing, offensive items, and leaf-smacking will see '
       'you triumph.'],
 418: ["Cute Bite-sized Platformer That Shouldn't Be Under-estimated As a "
       'sucker for pixel art an

['Very good metroidvania styled game.',
 'Has some challenging boss fights.',
 'Fantastic pixel art.',
 'It has some game mechanics just like Dark Souls .',
 'Great game with fantastic art and music.',
 'Dark Souls.',
 'So you have a melee attack and a ranged attack.',
 "I'm playing the game on normal difficulty and it's already quite challenging.",
 "It reminds me most of Hollow Knight, although it isn't quite as good as that game.",
 'I can say this game is worth of its full price.']